<a href="https://colab.research.google.com/github/ougrid/my-knowledge-resource/blob/master/CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:

from transformers import AutoProcessor, AutoModel
from PIL import Image
import torch
import requests

In [ ]:
model_name = 'google/siglip-base-patch16-256-multilingual'

In [ ]:
model = AutoModel.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float32)
processor = AutoProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
texts = ["แมว 2 ตัว", "ช้าง 4 ตัว"]

model_inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt").to(model.device)

In [ ]:
model_inputs

In [ ]:
with torch.no_grad():
    image_outputs = model.get_image_features(pixel_values = model_inputs['pixel_values'])
    text_outputs = model.get_text_features(input_ids = model_inputs['input_ids'])

In [ ]:
image_outputs.shape

torch.Size([1, 768])

In [ ]:
text_outputs.shape

torch.Size([2, 768])

In [ ]:
text_outputs

tensor([[-0.3581,  0.4364, -0.0470,  ..., -0.6224,  0.4730,  0.5909],
        [ 0.0120,  0.1631,  0.6020,  ..., -0.4624,  1.3735, -0.6439]],
       device='cuda:0')

In [ ]:
# normalized features
image_outputs = image_outputs / image_outputs.norm(p=2, dim=-1, keepdim=True)
text_outputs = text_outputs / text_outputs.norm(p=2, dim=-1, keepdim=True)

# cosine similarity as logits
similarity_per_text = torch.matmul(text_outputs, image_outputs.t().to(text_outputs.device))

In [ ]:
similarity_per_text

tensor([[ 0.1043],
        [-0.0237]], device='cuda:0')

In [ ]:
# probs = torch.sigmoid(logits_per_text)
for prob, text in zip(similarity_per_text, texts):
    print(f"{text}: {prob}")

แมว 2 ตัว: tensor([0.1043], device='cuda:0')
ช้าง 4 ตัว: tensor([-0.0237], device='cuda:0')


### With pipeline

In [ ]:
from transformers import pipeline

In [ ]:
image_classifier = pipeline(task="zero-shot-image-classification", model=model, tokenizer=processor.tokenizer, image_processor=processor.image_processor)

Device set to use cuda:0


In [ ]:
outputs = image_classifier(image, candidate_labels=texts)
for output in outputs:
    print(f"{output['label']}: {output['score']}")

แมว 2 ตัว: 0.19034600257873535
ช้าง 4 ตัว: 1.413200152455829e-07
